In [1]:
# app.py
import streamlit as st
import torch
from PIL import Image, ImageDraw
from torchvision import transforms
from facenet_pytorch import MTCNN
import numpy as np

# ======= MODELOS ==========
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Detector de caras
mtcnn = MTCNN(keep_all=True, device=device)

# Clasificador de emociones (cargá tu modelo entrenado)
modelo_emociones = torch.load('best_model.pth', map_location=device)
modelo_emociones.eval()

# Etiquetas del modelo FER-2013
emociones = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Preprocesamiento
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor()
])

# ======= APP STREAMLIT ==========
st.title("Detector de Emociones en Rostros")
st.write("Subí una imagen y detectamos las emociones de los rostros.")

uploaded_file = st.file_uploader("Elegí una imagen", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    img = Image.open(uploaded_file).convert('RGB')
    st.image(img, caption="Imagen original", use_column_width=True)

    # Detección de caras
    boxes, _ = mtcnn.detect(img)

    if boxes is not None:
        draw = ImageDraw.Draw(img)

        for box in boxes:
            # Recortar rostro
            cara = img.crop(box)
            cara_tensor = transform(cara).unsqueeze(0).to(device)  # [1, 1, 48, 48]

            # Clasificar emoción
            with torch.no_grad():
                output = modelo_emociones(cara_tensor)
                pred = torch.argmax(output, dim=1).item()
                emocion = emociones[pred]

            # Dibujar caja + emoción
            draw.rectangle(box.tolist(), outline='red', width=2)
            draw.text((box[0], box[1]-10), emocion, fill='red')

        # Mostrar resultado final
        st.image(img, caption="Detección con emociones", use_column_width=True)
    else:
        st.warning("No se detectaron rostros en la imagen.")


ModuleNotFoundError: No module named 'streamlit'